In [1]:
#code by arminnorouzi
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text

In [13]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("kaushal2896/english-to-german")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/english-to-german


In [75]:
eng_due_ds = tf.data.TextLineDataset(path + "/deu.txt")\
.map(lambda line: (tf.strings.split(line, '\t')[0],tf.strings.split(line, '\t')[1]))\
.shuffle(10000).prefetch(tf.data.AUTOTUNE)

In [76]:
dataset_size = eng_due_ds.cardinality().numpy()

# Now, you can use dataset_size to split the dataset.
train_size = int(0.8 * dataset_size)

train_dataset = eng_due_ds.take(train_size)
val_dataset = eng_due_ds.skip(train_size)

In [89]:
tokenizer = tensorflow_text.UnicodeScriptTokenizer()
for eng,due in train_dataset.batch(3).take(1):
    for eng_eg in eng.numpy():
        print(eng_eg.decode('utf-8'))
        print(tokenizer.tokenize(eng_eg.decode('utf-8')))

We apologize.
tf.Tensor([b'We' b'apologize' b'.'], shape=(3,), dtype=string)
That's cool.
tf.Tensor([b'That' b"'" b's' b'cool' b'.'], shape=(5,), dtype=string)
I kissed Tom.
tf.Tensor([b'I' b'kissed' b'Tom' b'.'], shape=(4,), dtype=string)


In [87]:
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())

[[b'What', b'you', b'know', b'you', b'can', b"'", b't', b'explain', b',', b'but', b'you', b'feel', b'it', b'.']]
